In [2]:
import pandas as pd
import plotnine as pn
import numpy as np

In [3]:
df = pd.read_csv("../build/tables/area_potential_offshore.csv")
df.head()

,shortname,area_km2,available area deep,available share deep,available area shallow,available share shallow
0,NOR,933027,894440,0.96,39996,0.04
1,GBR,731309,566440,0.77,167683,0.23
2,ESP,561763,489079,0.87,10162,0.02
3,ITA,536654,479622,0.89,36062,0.07
4,GRC,482910,445694,0.92,15171,0.03


In [36]:
vars_available_area = ["shortname", "available area deep", "available area shallow"]
available_area = df.loc[:, vars_available_area]
area_total = df.loc[:, ["shortname", "area_km2"]]

available_area = pd.melt(available_area, id_vars="shortname", value_vars=available_area.columns.drop("shortname"))
area_total = pd.melt(area_total, id_vars="shortname", value_vars=area_total.columns.drop("shortname"))
available_area.columns = ["Region", "Type", "Value"]
available_area

,Region,Type,Value
0,NOR,available area deep,894440
1,GBR,available area deep,566440
2,ESP,available area deep,489079
3,ITA,available area deep,479622
4,GRC,available area deep,445694
...,...,...,...
57,SVN,available area shallow,203
58,NOR/SWE,available area shallow,3
59,HRV/SVN,available area shallow,98
60,FRA/ITA,available area shallow,0


In [43]:
var_name = "Share of area"
var_unit = "-"
order = None


def stacked_bar_plot(df, x, y, fill, order=None, normalized=False, colors=None):
    r"""
    Plot a stacked bar plot.
    """
    _df = df.copy()

    if normalized:
        _df_max = _df.groupby(x).agg({y: "sum"}).reset_index()
        _df = _df.merge(_df_max, on=x, suffixes=("", "_max"))
        _df[y] = _df[y] / _df[y+"_max"]
        _df = _df.drop(columns=[y+"_max"])
        _df = _df.loc[~_df[y].isin([np.inf, -np.inf])]
    
    if colors:
        assert isinstance(colors, dict), "Colors should be a dictionary."
        assert _df.loc[:, fill].isin(colors.keys()).all(), "Not all colors are defined."
        color_map = pn.scale_fill_manual(breaks=list(colors.keys()), values=list(colors.values()))

    if order is not None:
        assert set(_df[x].unique()) == set(order), "Order should contain all items in x."
        categories = pd.Categorical(_df[x], categories=order, ordered=True)
        _df = _df.assign(**{x: categories})
    
    plot = (
        pn.ggplot(_df)

        + pn.geom_col(pn.aes(x=x, y=y, fill=fill))
        + pn.labs(x="Region", y=f"{var_name} ({var_unit})")
        + pn.scale_color_discrete(guide=False)
        + pn.theme(axis_text_x=pn.element_text(angle=90), legend_position="bottom")
        + color_map 
    )

    return plot

colors = {
    "available area deep": "#4c477d",
    "available area shallow": "#1aeaef",
}

order_by_sum = available_area.groupby("Region").agg({"Value": "sum"}).sort_values(by="Value", ascending=True).index

stacked_bar_plot(available_area, x="Region", y="Value", fill="Type", normalized=True, colors=colors, order=order_by_sum).save("rel.png", dpi=300, height=5, width=10, facecolor="w", transparent=False)

/home/jlauner/.conda/envs/jw4a-d2.1/lib/python3.11/site-packages/plotnine/ggplot.py:587: PlotnineWarning: Saving 10 x 5 in image.
/home/jlauner/.conda/envs/jw4a-d2.1/lib/python3.11/site-packages/plotnine/ggplot.py:588: PlotnineWarning: Filename: rel.png
/home/jlauner/.conda/envs/jw4a-d2.1/lib/python3.11/site-packages/plotnine/layer.py:344: PlotnineWarning: position_stack : Removed 2 rows containing missing values.
